In this notebook, me and my team, we are going to scrape an ecommerce website. We'll get into each individual product page and retrieve our information from there. This is the website we are going to scrape <a href="https://www.cdiscount.com/telephonie/v-144-0.html#nav=T%C3%A9l%C3%A9phonie_txt">Cdiscount</a>  – it's an French online shop.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Maintenant, nous allons définir l'URL de base de la page principale car nous en aurons besoin lorsque nous construirons nos URL pour chacun des produits individuels.

De plus, nous enverrons un agent utilisateur sur chaque requête HTTP, car si nous avons effectuez une requête GET à l'aide de requests , l'agent utilisateur est Python par défaut, ce qui peut être bloqué.

Donc, pour remplacer cela, nous allons déclarer une variable qui stockera notre user-agent.

In [47]:
baseurl = "https://www.cdiscount.com/telephonie/v-144-0.html#nav=T%C3%A9l%C3%A9phonie_txt"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

In [17]:
k = requests.get(baseurl, headers=headers).text
soup=BeautifulSoup(k,'html.parser')
productlist = soup.find_all("li",{"class":"cPdtItem jsPdtItem"})
soup


<!DOCTYPE html>

<html>
<head>
<title>Baleen - Error Page</title>
<meta content="width=device-width" name="viewport"/>
<meta charset="utf-8"/>
<link href="https://fonts.googleapis.com/css2?family=Roboto:wght@100;400&amp;display=swap" rel="stylesheet"/>
<link href="/.well-known/baleen/error-pages-in/assets/style.css" referrerpolicy="no-referrer" rel="stylesheet"/>
</head>
<body>
<div>
<h1 class="error-title">Error 403</h1>
<div class="error-message">
        You are not authorised to access this content
      </div>
<div class="error-illustration">
<!-- device_ok.svg -->
<svg version="1.1" viewbox="0 0 146 194" width="146px" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink">
<title>9B720660-BC66-468F-8F46-DA5D9A35113E</title>
<g fill="none" fill-rule="evenodd" id="ð«-Page-d'erreur" stroke="none" stroke-width="1">
<g id="Error-403" transform="translate(-257.000000, -403.000000)">
<g id="Group-4" transform="translate(257.000000, 403.631164)">
<g fill="#000000

In [14]:
r = requests.get(baseurl)

#if you want the whole content you can just do slicing stuff on the response stored in r or rather just do it with bs4

soup = BeautifulSoup(r.text, "html.parser")
s

[]

In [19]:
from selectorlib import Extractor
import requests 
import json 
from time import sleep

In [35]:
baseurl="https://www.amazon.com/dp/B07YDGHDFQ/ref=sspa_dk_detail_1?psc=1&pd_rd_i=B07YDGHDFQ&pd_rd_w=V8Vij&pf_rd_p=0c758152-61cd-452f-97a6-17f070f654b8&pd_rd_wg=xTOAJ&pf_rd_r=P8H8N568MSXRHR7H4DRH&pd_rd_r=bb5f00a6-6961-4b95-8810-dbb98dff2669&s=apparel&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUEzVERWRVJHUkRNOEpNJmVuY3J5cHRlZElkPUEwNDQyOTIxM0JaODZZMFI4SjhFOCZlbmNyeXB0ZWRBZElkPUEwMTE3NDc3MUhKRUZONFJVTzRaUSZ3aWRnZXROYW1lPXNwX2RldGFpbCZhY3Rpb249Y2xpY2tSZWRpcmVjdCZkb05vdExvZ0NsaWNrPXRydWU="

In [48]:
# Create an Extractor by reading from the YAML file
e = Extractor.from_yaml_file('selectors.yml')
def scrape(url):  

    headers = {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }

    # Download the page using requests
    print("Downloading %s"%url)
    r = requests.get(url, headers=headers)
    # Simple check to check if page was blocked (Usually 503)
    if r.status_code > 500:
        if "To discuss automated access to Amazon data please contact" in r.text:
            print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
        else:
            print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
        return None
    # Pass the HTML of the page and create 
    print(r.text)
    return e.extract(r.text)

In [49]:
scrape(baseurl)

<!DOCTYPE html>
<html>
  <head>
    <title>Baleen - Error Page</title>
    <meta content="width=device-width" name="viewport" />
    <meta charset="utf-8" />
    <link
      href="https://fonts.googleapis.com/css2?family=Roboto:wght@100;400&display=swap"
      rel="stylesheet"
    />
    <link href="/.well-known/baleen/error-pages-in/assets/style.css" rel="stylesheet" referrerpolicy="no-referrer"/>
  </head>

  <body>
    <div>
      <h1 class="error-title">Error 403</h1>
      <div class="error-message">
        You are not authorised to access this content
      </div>
      <div class="error-illustration">
        <!-- device_ok.svg -->
        <svg
          width="146px"
          viewBox="0 0 146 194"
          version="1.1"
          xmlns="http://www.w3.org/2000/svg"
          xmlns:xlink="http://www.w3.org/1999/xlink"
        >
          <title>9B720660-BC66-468F-8F46-DA5D9A35113E</title>
          <g
            id="ð«-Page-d'erreur"
            stroke="none"
            st

{'name': None,
 'price': None,
 'short_description': None,
 'images': None,
 'rating': None,
 'number_of_reviews': None,
 'variants': None,
 'product_description': None,
 'sales_rank': None,
 'link_to_all_reviews': None}